In [ ]:
pip install pytorch-tabnet

In [ ]:
 pip install imbalanced-learn

In [ ]:
import pandas as pd
import imblearn
from imblearn.over_sampling import ADASYN, SMOTE
from collections import Counter
#import umap
from sklearn.manifold import TSNE
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import FastICA, PCA, FactorAnalysis
from sklearn.metrics import roc_auc_score, recall_score, f1_score, accuracy_score,confusion_matrix, precision_score, log_loss, roc_curve
pd.set_option('display.max_columns', None)

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [ ]:
data_death = pd.read_csv('DeadMICE.csv')
data_death.head()

,Age,Gender,ethnicity,race,Fever,Cough,SOB,Fatigue,Sputum,Myalgia,Diarrhea,Nausea.or.vomiting,Sore.throat,Runny.nose.nasal.congestion,Loss.of.smell,Loss.of.taste,Headahce,Chest.discomfort..Chest.pain,Asymptomatic,smoking_history,hypertensionhx,diabeteshx,asthmahx,coronaryheartdiseasehx,copdhx,heartfailurehx,carcinomahx,immunosuppressionhx,ckdhx,ALT,CRP,D.dimer,Ferritin,HR,LDH,Lymphocytes,SpO2,Procalcitonin,RR,Systolic.BP,Temperature,Troponin,death
0,23,0,3,7,1,1,1,0,1,0,1,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,23,1.7,415,29.1,109,130,30.7,97,0.06,20,102,37.2,0.01,0
1,50,0,1,1,0,1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,21,5.6,150,153.2,108,130,9.2,94,0.08,16,127,37.3,0.01,0
2,29,0,1,7,1,1,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,10,1.2,625,42.4,88,133,15.1,97,0.07,19,103,36.9,0.01,0
3,57,1,2,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,34,12.2,206,373.9,67,133,15.4,98,0.05,18,153,37.4,0.01,0
4,26,0,2,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,51,14.3,273,107.7,100,147,18.3,96,0.43,14,120,37.0,0.01,0


In [ ]:
data_death.columns = data_death.columns.str.replace('.','_')
data_death = data_death.rename(columns={'Gender__female_0__male1_':'Gender'})
data_death.head()

,Age,Gender,ethnicity,race,Fever,Cough,SOB,Fatigue,Sputum,Myalgia,Diarrhea,Nausea_or_vomiting,Sore_throat,Runny_nose_nasal_congestion,Loss_of_smell,Loss_of_taste,Headahce,Chest_discomfort__Chest_pain,Asymptomatic,smoking_history,hypertensionhx,diabeteshx,asthmahx,coronaryheartdiseasehx,copdhx,heartfailurehx,carcinomahx,immunosuppressionhx,ckdhx,ALT,CRP,D_dimer,Ferritin,HR,LDH,Lymphocytes,SpO2,Procalcitonin,RR,Systolic_BP,Temperature,Troponin,death
0,23,0,3,7,1,1,1,0,1,0,1,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,23,1.7,415,29.1,109,130,30.7,97,0.06,20,102,37.2,0.01,0
1,50,0,1,1,0,1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,21,5.6,150,153.2,108,130,9.2,94,0.08,16,127,37.3,0.01,0
2,29,0,1,7,1,1,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,10,1.2,625,42.4,88,133,15.1,97,0.07,19,103,36.9,0.01,0
3,57,1,2,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,34,12.2,206,373.9,67,133,15.4,98,0.05,18,153,37.4,0.01,0
4,26,0,2,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,51,14.3,273,107.7,100,147,18.3,96,0.43,14,120,37.0,0.01,0


In [ ]:
X = data_death.drop(columns='death')
y= data_death['death']

In [ ]:
ada = ADASYN(random_state=0, n_neighbors=20)
X_res, y_res = ada.fit_resample(X, y)
print('Resampled dataset shape {}'.format(Counter(y_res)))

Resampled dataset shape Counter({0: 878, 1: 864})


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
#reducer = umap.UMAP()
transformer = FastICA(random_state=0)
X_transformed = transformer.fit_transform(X_res)
X_transformed.shape

(1742, 42)

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X_transformed,y_res, test_size=0.1, random_state=0)

In [ ]:
len(y_test[y_test == 0])

96

In [ ]:
data_death.head()

,Age,Gender,ethnicity,race,Fever,Cough,SOB,Fatigue,Sputum,Myalgia,Diarrhea,Nausea_or_vomiting,Sore_throat,Runny_nose_nasal_congestion,Loss_of_smell,Loss_of_taste,Headahce,Chest_discomfort__Chest_pain,Asymptomatic,smoking_history,hypertensionhx,diabeteshx,asthmahx,coronaryheartdiseasehx,copdhx,heartfailurehx,carcinomahx,immunosuppressionhx,ckdhx,ALT,CRP,D_dimer,Ferritin,HR,LDH,Lymphocytes,SpO2,Procalcitonin,RR,Systolic_BP,Temperature,Troponin,death
0,23,0,3,7,1,1,1,0,1,0,1,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,23,1.7,415,29.1,109,130,30.7,97,0.06,20,102,37.2,0.01,0
1,50,0,1,1,0,1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,21,5.6,150,153.2,108,130,9.2,94,0.08,16,127,37.3,0.01,0
2,29,0,1,7,1,1,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,10,1.2,625,42.4,88,133,15.1,97,0.07,19,103,36.9,0.01,0
3,57,1,2,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,34,12.2,206,373.9,67,133,15.4,98,0.05,18,153,37.4,0.01,0
4,26,0,2,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,51,14.3,273,107.7,100,147,18.3,96,0.43,14,120,37.0,0.01,0


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_transformed, y_res, 
                                                        test_size=0.1, random_state=10)
target = data_death['death']
skf = StratifiedKFold(n_splits=5)
fold_no = 1
for train_index, test_index in skf.split(data_death, target):
  train = data_death.loc[train_index,:]
  test = data_death.loc[test_index,:]
  print('Fold',str(fold_no),'Class Ratio:',sum(test['death'])/len(test['death']))
  fold_no += 1
def train_model(train, test, fold_no):
  
  
  clf = TabNetClassifier(n_d=64,n_shared=2,mask_type='sparsemax', momentum=0.3,n_steps=3,n_independent=2,lambda_sparse=0.003,gamma=2.8)
  clf.fit(
    X_train=X_train, y_train=y_train,
    eval_set=[(X_train, y_train), (X_test, y_test)],
    eval_name=['train', 'test'],
    eval_metric=['auc'], max_epochs=150, batch_size=128, patience=60
  
  )
  predictions = clf.predict(X_test)
  #print('Fold',str(fold_no),'Accuracy:',recall_score(y_test,predictions))
  print('Roc score is',np.mean(roc_auc_score(y_test,predictions)))
  print('f1 score is'np.mean(f1_score(y_test,predictions)))
  print('accuracy score is'np.mean(accuracy_score(y_test,predictions)))
  print('recall score is'np.mean(recall_score(y_test,predictions)))
  print('precision score is'np.mean(precision_score(y_test,predictions))) 
import numpy as np

fold_no = 1
for train_index, test_index in skf.split(data_death, target):
  train = data_death.loc[train_index,:]
  test = data_death.loc[test_index,:]
  train_model(train,test,fold_no)
  fold_no += 1


In [ ]:
tab_pred = clf.predict(X_test)

In [ ]:
# plot auc
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import files

plt.plot(clf.history['train_auc'],  label='Training accuracy')
plt.plot(clf.history['test_auc'],label='Validation accuracy')
plt.rcParams["figure.figsize"]=(10, 10)
plt.title('Accuracy score for training and testing')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.savefig('accdead.pdf', transparent=True,dphi=300)
files.download("accdead.pdf")

In [ ]:
# plot losses
import matplotlib.pyplot as plt
plt.plot(clf.history['loss'])
plt.rcParams["figure.figsize"]=(10, 10)
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.savefig('lossdead.pdf', transparent=True,dphi=300)
files.download("lossdead.pdf");

In [ ]:
import numpy as np
Rf = RandomForestClassifier()
Rf.fit(X_train, y_train)
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import files

def plot_feature_importance(importance,names,model_type):


  #Create arrays from feature importance and feature names
  feature_importance = np.array(importance)
  feature_names = np.array(names)

  #Create a DataFrame using a Dictionary
  data_icu={'feature_names':feature_names,'feature_importance':feature_importance}
  fi_df = pd.DataFrame(data_icu)

  #Sort the DataFrame in order decreasing feature importance
  fi_df.sort_values(by=['feature_importance'], ascending=False,inplace=True,)

  #Define size of bar plot
  plt.figure(figsize=(11,11))
  #Plot Searborn bar chart
  sns.barplot(x=fi_df['feature_importance'], y=fi_df['feature_names'])
  #Add chart labels
  plt.title('FEATURE IMPORTANCE')
  plt.xlabel('FEATURE IMPORTANCE')
  plt.ylabel('FEATURES')

In [ ]:
plot_feature_importance(clf.feature_importances_, X.columns,'Tabnet')

In [ ]:
clf_probs = clf.predict_proba(X_test)
clf_probs = clf_probs[:, 1]


In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc

In [ ]:
clf_precision, clf_recall, _ = precision_recall_curve(y_test, clf_probs)
clf_f1, clf_auc = f1_score(y_test, tab_pred), auc(clf_recall, clf_precision)

In [ ]:
Adasyn_icu = len(y_test[y_test==1]) / len(y_test)

In [ ]:
plt.plot(clf_recall, clf_precision, marker='.',)
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve for Proposed Model')
# show the legend
plt.legend()
# show the plot

plt.savefig('precalldead.pdf', transparent=True,dphi=300)
files.download("precalldead.pdf");

In [ ]:
explain_matrix, masks = clf.explain(X_test)

In [ ]:
from google.colab import files

In [ ]:
import matplotlib.pyplot as plt
fig, axis = plt.subplots(2,1, figsize=(15,15))
for i in range(2):
  axis[i].imshow(masks[i][:20])
  axis[i].set_title(f'masks{i}')
  fig.suptitle('Feature Importance Masks')
plt.savefig('masksdead.pdf', transparent=True,dphi=300)
files.download("masksdead.pdf")   

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
cm = confusion_matrix(y_test, tab_pred)
cmd = ConfusionMatrixDisplay(cm, display_labels=['Death','No Death'])
cmd.plot()
plt.savefig('cmdead.pdf', transparent=True,dphi=300)
files.download("cmdead.pdf")   